**Load the HDF5 File**

In [7]:
import os
import h5py
import numpy as np
import torch
from torch import Tensor
from torch.utils.data import Dataset, DataLoader, random_split
from torch_geometric.data import Data
from typing import Callable, List, Optional

In [30]:
class H5PointCloudDataset(Dataset):
    def __init__(self, file_path, transform=None):
        # Open the hdf5 file
        with h5py.File(file_path, 'r') as f:
            # Load the entire dataset into memory
            self.data = np.array(f['data'])    # Shape: (num_blocks, 4096, 9)
            self.labels = np.array(f['label'])   # Shape: (num_blocks, 4096)
        self.transform = transform

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self,idx):
        xs: List[Tensor] = []
        ys: List[Tensor] = []
        # Retrieve the point cloud block and corresponding labels
        xs += torch.from_numpy(self.data[idx][:]).unbind(0)
        ys += torch.from_numpy(self.labels[idx][:]).to(torch.long).unbind(0)
        data_list = []
        for i, (x, y) in enumerate(zip(xs, ys)):
            data = Data(pos=x[:, :3], x=x[:, 3:], y=y)  
            data_list.append(data)
        
        return data_list

In [31]:
# File path to the hdf5 dataset
script_dir = os.path.dirname(os.getcwd())
hdf5_file_path= os.path.join(script_dir, '.', 'docs', 'sim_pc_dataset.h5')

# Create the dataset
full_dataset = H5PointCloudDataset(hdf5_file_path)

# Define split sizes (e.g., 80% training and 20% validation)
total_size = len(full_dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

# Randomly split the dataset
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

In [33]:
print(train_dataset[1])

[Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos=[4096, 3]), Data(x=[4096, 6], y=[4096], pos

Data(x=[4096, 6], y=[4096], pos=[4096, 3])

Data(x=[4096, 6], y=[4096], pos=[4096, 3])

In [32]:

# Assuming 'dataset' is your PyTorch Geometric dataset
index = 1  # Index of the data instance you want to extract

# Extract the data instance from the dataset
data_instance = train_dataset[index]

# Access the features, labels, and positions of this instance
x = data_instance.x
y = data_instance.y
pos = data_instance.pos

# Print these values
print("Features (x):", x)
print("Labels (y):", y)
print("Positions (pos):", pos)
print("Center Point:", torch.mean(pos, dim=0))

print(len(x))


AttributeError: 'list' object has no attribute 'x'

In [40]:
train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True,
                          num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=12, shuffle=False,
                         num_workers=8)

**Training with PointNet2**

In [41]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import MLP, PointNetConv, fps, global_max_pool, radius
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MLP, knn_interpolate
from torch_geometric.typing import WITH_TORCH_CLUSTER
from torch_geometric.data import Data

from sklearn.linear_model import RANSACRegressor
from scipy.spatial import ConvexHull

if not WITH_TORCH_CLUSTER:
    quit("This example requires 'torch-cluster'")

In [42]:
class SAModule(torch.nn.Module):
    def __init__(self, ratio, r, nn):
        super().__init__()
        self.ratio = ratio
        self.r = r
        self.conv = PointNetConv(nn, add_self_loops=False)

    def forward(self, x, pos, batch):
        idx = fps(pos, batch, ratio=self.ratio)
        row, col = radius(pos, pos[idx], self.r, batch, batch[idx],
                          max_num_neighbors=64)
        edge_index = torch.stack([col, row], dim=0)
        x_dst = None if x is None else x[idx]
        x = self.conv((x, x_dst), (pos, pos[idx]), edge_index)
        pos, batch = pos[idx], batch[idx]
        return x, pos, batch


class GlobalSAModule(torch.nn.Module):
    def __init__(self, nn):
        super().__init__()
        self.nn = nn

    def forward(self, x, pos, batch):
        x = self.nn(torch.cat([x, pos], dim=1))
        x = global_max_pool(x, batch)
        pos = pos.new_zeros((x.size(0), 3))
        batch = torch.arange(x.size(0), device=batch.device)
        return x, pos, batch

In [43]:
class FPModule(torch.nn.Module):
    def __init__(self, k, nn):
        super().__init__()
        self.k = k
        self.nn = nn

    def forward(self, x, pos, batch, x_skip, pos_skip, batch_skip):
        x = knn_interpolate(x, pos, pos_skip, batch, batch_skip, k=self.k)
        if x_skip is not None:
            x = torch.cat([x, x_skip], dim=1)
        x = self.nn(x)
        return x, pos_skip, batch_skip

In [44]:
class Net(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        # Input channels account for both `pos` and node features.
        self.sa1_module = SAModule(0.2, 0.2, MLP([3 + 6, 64, 64, 128])) # 3 (pos) + 6 (x)
        self.sa2_module = SAModule(0.25, 0.4, MLP([128 + 3, 128, 128, 256]))
        self.sa3_module = GlobalSAModule(MLP([256 + 3, 256, 512, 1024]))

        self.fp3_module = FPModule(1, MLP([1024 + 256, 256, 256]))
        self.fp2_module = FPModule(3, MLP([256 + 128, 256, 128]))
        self.fp1_module = FPModule(3, MLP([128 + 6, 128, 128, 128]))

        self.mlp = MLP([128, 128, 128, num_classes], dropout=0.5, norm=None)

        self.lin1 = torch.nn.Linear(128, 128)
        self.lin2 = torch.nn.Linear(128, 128)
        self.lin3 = torch.nn.Linear(128, num_classes)

    def forward(self, data):
        sa0_out = (data.x, data.pos, data.batch)
        sa1_out = self.sa1_module(*sa0_out)
        sa2_out = self.sa2_module(*sa1_out)
        sa3_out = self.sa3_module(*sa2_out)

        fp3_out = self.fp3_module(*sa3_out, *sa2_out)
        fp2_out = self.fp2_module(*fp3_out, *sa1_out)
        x, _, _ = self.fp1_module(*fp2_out, *sa0_out)

        return self.mlp(x).log_softmax(dim=-1)

In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(num_classes=13).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train():
    model.train()

    total_loss = correct_nodes = total_nodes = 0
    for i, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct_nodes += out.argmax(dim=1).eq(data.y).sum().item()
        total_nodes += data.num_nodes

        if (i + 1) % 10 == 0:
            print(f'[{i+1}/{len(train_loader)}] Loss: {total_loss / 10:.4f} '
                  f'Train Acc: {correct_nodes / total_nodes:.4f}')
            total_loss = correct_nodes = total_nodes = 0

In [46]:
from torchmetrics.functional import jaccard_index
@torch.no_grad()
def test(loader):
    model.eval()
    ious = []
    
    for data in loader:
        data = data.to(device)
        outs = model(data)

        sizes = (data.ptr[1:] - data.ptr[:-1]).tolist()
        for out, y in zip(outs.split(sizes), data.y.split(sizes)):            
            iou = jaccard_index(out.argmax(dim=-1), y, 
                                num_classes=loader.dataset.num_classes, 
                                task="multiclass")
            ious.append(iou)        

    iou = torch.tensor(ious, device=device)

    mean_iou = iou.mean()
    return float(mean_iou)


In [47]:
for epoch in range(1, 21):
    train()
    iou = test(test_loader)
    print(f'Epoch: {epoch:02d}, Test IoU: {iou:.4f}')

In [ ]:
# Check block and stride size from S3DIS file


import h5py
import numpy as np

def get_block_size(h5_file):
    block_sizes = []
    for block_name in h5_file.keys():
        data = h5_file[block_name]['data'][:]  # Shape: [N, 9] (XYZ, RGB, normalized XYZ)
        xyz = data[:, :3]  # Original coordinates (not normalized)
        
        # Compute spatial extent (block size)
        min_coords = np.min(xyz, axis=0)
        max_coords = np.max(xyz, axis=0)
        block_size = max_coords[:2] - min_coords[:2]  # Use X and Y dimensions (ignore Z)
        block_sizes.append(block_size)
    
    avg_block_size = np.mean(block_sizes, axis=0)
    print(f"Average block size (X, Y): {avg_block_size} meters")
    return avg_block_size

data_file_path = r"C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\AI\scripts\datasets\s3dis\raw\ply_data_all_0.h5"
with h5py.File(data_file_path, 'r') as f:
    avg_block_size = get_block_size(f)
    print(avg_block_size)

In [34]:
import h5py
f = h5py.File(data_file_path, 'r') #train_pc_path, data_file_path
list(f.keys())

['data', 'label']

In [35]:
dset = f['data']
dset.shape

(1000, 4096, 9)

In [36]:
dlabel = f['label']
dlabel.shape

(1000, 4096)

In [28]:
print(f['data'][0][0])

[0.153      0.478      0.051      0.4        0.39215687 0.34509805
 0.12577319 0.22026333 0.01600753]


In [22]:
def infer_stride(h5_file):
    origins = []
    for block_name in h5_file.keys():
        data = h5_file[block_name]['data'][:]
        xyz = data[:, :3]
        min_coords = np.min(xyz, axis=0)
        origins.append(min_coords[:2])  # Track X,Y origins
    
    # Sort origins by X then Y
    origins_sorted = sorted(origins, key=lambda x: (x[0], x[1]))
    
    # Compute pairwise distances between consecutive blocks
    strides = []
    for i in range(1, len(origins_sorted)):
        dx = origins_sorted[i][0] - origins_sorted[i-1][0]
        dy = origins_sorted[i][1] - origins_sorted[i-1][1]
        strides.extend([dx, dy])
    
    # Find most common stride (assuming fixed stride)
    stride_x, stride_y = np.median(strides), np.median(strides)
    print(f"Inferred stride (X, Y): {stride_x}, {stride_y} meters")
    return stride_x, stride_y

with h5py.File(data_file_path, 'r') as f:
    stride_x, stride_y = infer_stride(f)

ValueError: Field names only allowed for compound types

In [ ]:
checkpoint_path = ".\checkpoints\pointnet2_s3dis_transform_seg_x6_27_checkpoint.pth"  # Path to save the checkpoint

# Save model, optimizer state, and any other info needed
torch.save({
    'epoch': 27,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    #'loss': loss,
    #'test_accuracy': test_acc
}, checkpoint_path)

print("Checkpoint saved successfully!")

In [ ]:
# Apply LoRA to segmentation head
model.seg_head = lora.Linear(...)  # See earlier LoRA code

# Train
for epoch in range(epochs):
    for data, labels in train_loader:
        ...

In [ ]:
import torch.nn as nn
import loralib as lora

# Load the original model
model = torch.load("original_model.pth")

# Freeze the entire base model
for param in model.parameters():
    param.requires_grad = False

# Replace specific layers with LoRA-enabled layers
# Example: Replace a linear layer in the segmentation head
model.seg_layer[0] = lora.Linear(
    in_features=128,        # Original input dimension
    out_features=64,       # Original output dimension
    r=8,                   # Rank of the low-rank matrices (hyperparameter)
    lora_alpha=16,         # Scaling factor for LoRA weights
    merge_weights=False     # Keep LoRA separate during training
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define LoRA-compatible optimizer (only train LoRA params)
optimizer = torch.optim.Adam(
    lora.lora_parameters(model),  # Only LoRA parameters
    lr=1e-3,                      # Higher LR for adapter training
    weight_decay=0.01
)

# Training loop (similar to standard fine-tuning)
for epoch in range(num_epochs):
    model.train()
    for points, labels in train_loader:
        points, labels = points.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(points)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [ ]:
torch.save(lora.lora_state_dict(model), "pointcloud_lora_weights.pth")

In [ ]:
# Load the original model and LoRA weights
model = torch.load("original_model.pth")
lora_weights = torch.load("pointcloud_lora_weights.pth")

# Merge LoRA weights into the base model
model.load_state_dict(lora_weights, strict=False)
lora.mark_only_lora_as_trainable(model)  # Optional: Revert if needed later

# Now use `model` as a standard fine-tuned model

In [ ]:
import torch
from pointnet2 import PointNet2Seg
import loralib as lora

class PointNet2LoRA(PointNet2Seg):
    def __init__(self, num_classes, r=8, alpha=16):
        super().__init__(num_classes)
        
        # Freeze base model
        for param in self.parameters():
            param.requires_grad = False
        
        # Replace final MLP layers with LoRA
        self.seg_layers = nn.Sequential(
            lora.Linear(256, 128, r=r, lora_alpha=alpha),
            nn.ReLU(),
            lora.Linear(128, num_classes, r=r, lora_alpha=alpha)
        )

# Usage
model = PointNet2LoRA(num_classes=10, r=8)